# [Nicholas Yim, Aseef Durrani]
# Dataset \#3 - Energy Consumption Clustering
---

In [840]:
import pandas as pd
import numpy as np
import plotly.express as px

# **A. Load and Explore Dataset**

In [841]:
# Load the dataset
data = pd.read_csv("../datasets/energy/owid-energy-data.csv")

# Preview of the dataset
print("Preview of the dataset:")
print(data.head())

# Number of samples and features
print("\nDataset shape (samples, features):", data.shape)

Preview of the dataset:
         country  year iso_code  population  gdp  biofuel_cons_change_pct  \
0  ASEAN (Ember)  2000      NaN         NaN  NaN                      NaN   
1  ASEAN (Ember)  2001      NaN         NaN  NaN                      NaN   
2  ASEAN (Ember)  2002      NaN         NaN  NaN                      NaN   
3  ASEAN (Ember)  2003      NaN         NaN  NaN                      NaN   
4  ASEAN (Ember)  2004      NaN         NaN  NaN                      NaN   

   biofuel_cons_change_twh  biofuel_cons_per_capita  biofuel_consumption  \
0                      NaN                      NaN                  NaN   
1                      NaN                      NaN                  NaN   
2                      NaN                      NaN                  NaN   
3                      NaN                      NaN                  NaN   
4                      NaN                      NaN                  NaN   

   biofuel_elec_per_capita  ...  solar_share_elec  solar

In [842]:
# Load the codebook and define reference data
codebook = pd.read_csv("../datasets/energy/owid-energy-codebook.csv")
index_cols = ["country", "year", "iso_code"]
deprecated_countries = ["burma", "macedonia", "swaziland", "czech republic"]

# Validation Functions
def validate_columns_in_codebook(data, codebook):
    """Check if all columns in the dataset are documented in the codebook.
    This ensures data completeness and proper documentation."""
    missing_in_codebook = data.columns.difference(codebook["column"])
    if not missing_in_codebook.empty:
        print(f"Columns not in codebook: {missing_in_codebook.tolist()}")

def validate_codebook_columns_in_data(data, codebook):
    """Check if all columns mentioned in the codebook exist in the dataset.
    This identifies any missing or renamed columns."""
    missing_in_data = set(codebook["column"]) - set(data.columns)
    if missing_in_data:
        print(f"Codebook columns missing in data: {list(missing_in_data)}")

def validate_column_whitespace(data):
    """Check for whitespace in column names, which can cause issues in data processing
    and should be replaced with underscores for consistency."""
    columns_with_space = [col for col in data.columns if " " in col]
    if columns_with_space:
        print(f"Columns with whitespace: {columns_with_space}")

def validate_column_case(data):
    """Check for uppercase characters in column names.
    Column names should be lowercase for consistency."""
    non_lowercase_columns = [col for col in data.columns if col != col.lower()]
    if non_lowercase_columns:
        print(f"Columns with uppercase characters: {non_lowercase_columns}")

def validate_non_nan_rows(data, index_cols):
    """Check for rows that contain only NaN values (excluding index columns).
    These rows provide no useful information and should be removed."""
    nan_rows = data.drop(columns=index_cols, errors="ignore").isnull().all(axis=1)
    if nan_rows.any():
        print(f"Rows with all NaN values: {nan_rows.sum()}")

def validate_deprecated_country_names(data, deprecated_countries):
    """Check for deprecated country names that should be updated to their current names
    for consistency and accuracy."""
    countries_in_data = data["country"].str.lower().unique()
    for country in deprecated_countries:
        if country in countries_in_data:
            print(f"Deprecated country found: {country}")

# Run all validations
print("\nValidating dataset...")
validate_columns_in_codebook(data, codebook)
validate_codebook_columns_in_data(data, codebook)
validate_column_whitespace(data)
validate_column_case(data)
validate_non_nan_rows(data, index_cols)
validate_deprecated_country_names(data, deprecated_countries)
print("Dataset validation complete.")


Validating dataset...
Dataset validation complete.


In [843]:
# Fraction of missing values
missing_fraction = data.isnull().sum() / len(data)
missing_fraction = missing_fraction[missing_fraction > 0].sort_values(ascending=False)

# Display missing values
print("\nColumns with missing values and their fractions:")
print(missing_fraction)

# Visualize missing values
missing_df = missing_fraction.reset_index()
missing_df.columns = ["Feature", "Missing Fraction"]
fig = px.bar(
    missing_df,
    x="Feature",
    y="Missing Fraction",
    title="Fraction of Missing Values per Feature",
    labels={"Feature": "Feature", "Missing Fraction": "Missing Fraction"},
)
fig.update_layout(xaxis=dict(tickangle=45), width=900)
fig.show()


Columns with missing values and their fractions:
biofuel_cons_change_pct    0.917202
solar_cons_change_pct      0.897717
biofuel_cons_per_capita    0.889969
nuclear_cons_change_pct    0.886209
wind_cons_change_pct       0.885017
                             ...   
iso_code                   0.229232
gas_production             0.227994
oil_prod_change_twh        0.210389
oil_production             0.198606
population                 0.154273
Length: 128, dtype: float64


In [844]:
# Identify and analyze categorical variables
categorical_variables = data.select_dtypes(include=["object"]).columns
print("\nCategorical Variables:")
print(categorical_variables.tolist())

# Visualize count of unique countries
fig = px.bar(
    x=data["country"].value_counts().index,
    y=data["country"].value_counts().values,
    title="Number of Records per Country",
    labels={"x": "Country", "y": "Number of Records"},
)
fig.update_layout(width=1000, xaxis=dict(tickangle=45))
fig.show()


Categorical Variables:
['country', 'iso_code']


In [845]:
# Summary statistics for numerical features
print("\nSummary statistics for numerical features:")
print(data.describe())

# Normalize numerical features manually for better visualization
numerical_features = data.select_dtypes(include=[np.number]).columns
normalized_data = data[numerical_features].apply(lambda x: (x - x.min()) / (x.max() - x.min()), axis=0)

# Create a summary DataFrame for normalized statistics
summary_df = normalized_data.describe().reset_index().melt(id_vars="index")

# Improved Plotly scatter plot with clearer y-axis
fig = px.scatter(
    summary_df,
    x="variable",
    y="value",
    color="index",
    title="Summary Statistics for Numerical Features (Normalized)",
    labels={"variable": "Feature", "value": "Normalized Value", "index": "Statistic"},
)
fig.update_layout(
    xaxis=dict(tickangle=45, title="Features"),
    yaxis=dict(title="Normalized Values (0 to 1)", range=[0, 1]),
    width=1200,
    height=600,
    legend_title="Statistic",
    template="plotly_white",  # Add a clean background style
    title_font=dict(size=20),  # Enhance title size
)
fig.show()


Summary statistics for numerical features:
               year    population           gdp  biofuel_cons_change_pct  \
count  21812.000000  1.844700e+04  1.177500e+04              1806.000000   
mean    1974.195718  1.054051e+08  4.260596e+11                45.489759   
std       35.342860  4.665375e+08  3.508591e+12               266.131064   
min     1900.000000  1.833000e+03  1.642060e+08              -100.000000   
25%     1946.000000  1.714291e+06  1.438637e+10                -0.500000   
50%     1984.000000  6.998022e+06  4.393385e+10                 8.189000   
75%     2004.000000  2.571993e+07  1.830838e+11                26.550000   
max     2023.000000  8.045311e+09  1.301126e+14              5659.328000   

       biofuel_cons_change_twh  biofuel_cons_per_capita  biofuel_consumption  \
count              2796.000000              2400.000000          2876.000000   
mean                  2.867027               136.600523            39.082519   
std                  10.692769 

In [846]:
# Histograms for selected features
selected_features = [
    "population", "gdp", "energy_per_capita", 
    "fossil_share_energy", "renewables_share_energy"
]

# Plot histograms for selected features
for feature in selected_features:
    fig = px.histogram(
        data, 
        x=feature, 
        nbins=20, 
        title=f"Histogram of {feature}",
        labels={feature: feature, "count": "Frequency"},
    )
    fig.show()

In [847]:
# Distribution of records over years
fig = px.histogram(
    data, 
    x="year", 
    title="Distribution of Records Over Years", 
    labels={"year": "Year", "count": "Frequency"},
    nbins=20,
)
fig.update_layout(bargap=0.1)
fig.show()

# **Dataset Exploration**

## Overview
This notebook explores the **Our World in Data Energy Consumption Dataset** by loading the data, validating its structure, and visualizing key aspects. The exploration focuses on understanding the dataset's distribution, identifying missing values, and summarizing numerical features to prepare it for further analysis.

---

### **Steps Performed**

#### **1. Dataset Validation**
- **Purpose**: To ensure the dataset is clean, consistent, and ready for analysis.
- **Validation Checks**:
  - Columns in the dataset match the codebook.
  - All columns in the dataset are lowercase and do not contain whitespace.
  - Deprecated country names (e.g., "Swaziland", "Burma") are not present.
  - Rows contain at least one non-NaN value.
- **Outcome**: No critical issues were identified during validation, ensuring the dataset's consistency.

---

#### **2. Missing Value Analysis**
- **Purpose**: To identify features with missing data for potential cleaning or imputation.
- **Key Findings**:
  - Many columns have a high fraction of missing values (>80%).
  - Missing data is visualized using a bar chart for clarity.

---

#### **3. Dataset Statistics**
- **Purpose**: To summarize numerical features and provide an overview of data ranges.
- **Key Insights**:
  - Features such as `population`, `gdp`, and `energy_per_capita` exhibit large variability.
  - Summary statistics (mean, median, std, min, max) help in understanding data distributions.

---

#### **4. Feature Exploration**
- **Purpose**: To visualize key features and their distributions.
- **Steps**:
  - Histograms are plotted for selected features (`population`, `gdp`, `energy_per_capita`, `fossil_share_energy`, `renewables_share_energy`) to reveal their distributions.
  - Insights on feature skewness or sparsity are derived.
  - The distribution of records over years and by country is visualized to understand temporal and geographical coverage.

---

### **Visualizations and Insights**
1. **Missing Value Distribution**:
   - A bar chart highlights columns with the highest fraction of missing values.
   - Columns with >80% missing values may need to be dropped or imputed.

2. **Numerical Feature Summary**:
   - A scatter plot visualizes normalized summary statistics (mean, min, max, etc.) for all numerical features.
   - Reveals potential outliers or irregular ranges.

3. **Histograms**:
   - Histograms for key features such as `population` and `energy_per_capita` reveal significant skewness, indicating the presence of outliers or imbalanced distributions.

4. **Temporal Analysis**:
   - The number of records per year is consistent, with a drop in recent years indicating incomplete data for later periods.

5. **Geographical Analysis**:
   - The distribution of records across countries is imbalanced, with certain countries contributing more data than others.

---

## Insights
- The dataset requires careful handling of missing values and imbalances in temporal and geographical coverage.
- Visualizations provide a clear understanding of key features and their distributions, helping guide subsequent clustering or modeling efforts.

# **B. Pre-Processing of Dataset**

In [848]:
# Filter data from the year 2002 to 2021
data_2002_2021 = data[(data["year"] >= 2002) & (data["year"] <= 2021)].copy()

print(f"Data filtered to include records from the year 2002 to 2021. New shape: {data_2002_2021.shape}")

Data filtered to include records from the year 2002 to 2021. New shape: (5681, 130)


## Data Filtering Design: 2002-2021 Time Period Selection

### Overview
We filtered the dataset to include only records from 2002 to 2021, creating a 20-year window that can be evenly divided into four 5-year periods for clustering analysis:
- Period 1: 2002-2006
- Period 2: 2007-2011
- Period 3: 2012-2016
- Period 4: 2017-2021

### Rationale

1. **Even Period Distribution**
   - The 20-year span divides perfectly into four 5-year periods
   - Allows for consistent comparison across time periods
   - Each period has equal weight in the analysis

2. **Data Completeness**
   - More recent data tends to be more complete and reliable
   - Most countries have consistent reporting during this timeframe
   - Removes us haveing to impute missing data

3. **Contemporary Relevance**
   - Captures modern energy trends and transitions
   - Includes the rise of renewable energy technologies
   - Reflects current global energy policies and practices

4. **Temporal Analysis**
   - 5-year averages help smooth out year-to-year fluctuations
   - Four periods provide enough data points to observe trends
   - Allows for meaningful comparison of cluster evolution over time

### Benefits
- Reduces noise from annual variations
- Provides stable clustering results for each period
- Enables analysis of how country clusters evolve over time
- Balances data availability with temporal resolution

In [849]:
# List of continents and keywords to exclude, as well as specific countries missing data/minimal influence
exclude_keywords = [
    "income", "middle east", "middle africa", "oecd", "opec", 
    "persian gulf", "south and central america", "asean", "wake island",
    "pacific islands", "territories", "western africa", "world", "g7", "g20", "cis", 
    "africa", "antarctica", "asia", "europe", "north america", "south america", "oceania",
    "latin america", "eastern africa", "africa (ei)", "(EIA)", "(Shift)", "Serbia and Montenegro", 
    "Tuvalu", "Micronesia (country)", "Northern Mariana Islands", "Micronesia (country)", 
    "Curacao", "Netherlands Antilles","East Timor","Central America", "Micronesia", 
    "South Sudan", "Montenegro", "Western Sahara", "Niue"
]

# Convert keywords to lowercase for case-insensitive matching
exclude_keywords_lower = [kw.lower() for kw in exclude_keywords]

# Filter out unwanted regions, continents, and specific cases
data_2002_2021 = data_2002_2021[
    ~(
        data_2002_2021["country"].str.lower().str.contains("|".join(exclude_keywords_lower), na=False) |
        (data_2002_2021["country"].str.strip().str.lower() == "africa")  # Exclude exact "Africa"
    )
]

print(f"Data shape after filtering excluded regions/categories: {data_2002_2021.shape}")

# Retain only the latest record for each country (based on the assumption that 'year' exists and is chronological)
filtered_data_unique = (
    data_2002_2021.sort_values("year", ascending=False)
    .drop_duplicates(subset=["country"], keep="first")
)

# Retain only the oldest record for each country (based on the assumption that 'year' exists and is chronological)
filtered_data_unique2 = (
    data_2002_2021.sort_values("year", ascending=False)
    .drop_duplicates(subset=["country"], keep="last")
)

print("\n\n\nFiltered data unique 2: ", data_2002_2021.sort_values("year", ascending=False)
    .drop_duplicates(subset=["country"], keep="last"))

# Save the resulting dataset to a CSV file
filtered_data_unique.to_csv("../datasets/energy/analysis/filtered_cleaned_data_unique.csv", index=False)

# Save the resulting dataset to a CSV file
filtered_data_unique2.to_csv("../datasets/energy/analysis/filtered_cleaned_data_unique2.csv", index=False)

# Print the unique list of remaining countries
remaining_countries = filtered_data_unique["country"].unique()
print("Remaining countries after filtering:")
for country in sorted(remaining_countries):
    print(country)

Data shape after filtering excluded regions/categories: (4160, 130)



Filtered data unique 2:            country  year iso_code  population           gdp  \
21091     Vietnam  2002      VNM  80642304.0  2.506834e+11   
20756     Uruguay  2002      URY   3306446.0  3.725087e+10   
20800  Uzbekistan  2002      UZB  25579032.0  1.227578e+11   
21790    Zimbabwe  2002      ZWE  11984643.0  2.487248e+10   
21450       Yemen  2002      YEM  19660662.0  7.858648e+10   
...           ...   ...      ...         ...           ...   
12959     Namibia  2002      NAM   1888531.0  1.202187e+10   
12925     Myanmar  2002      MMR  46480236.0  1.015915e+11   
12801  Mozambique  2002      MOZ  18694948.0  2.102537e+10   
12677     Morocco  2002      MAR  29301820.0  1.513858e+11   
10567  Kyrgyzstan  2002      KGZ   5026646.0  1.617050e+10   

       biofuel_cons_change_pct  biofuel_cons_change_twh  \
21091                      NaN                      NaN   
20756                      NaN           

/var/folders/kb/ldyhzd_s2sg_8j8sr3c0f23h0000gn/T/ipykernel_95786/1225751919.py:19: UserWarning:

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.



In [850]:
# For the year 2002 dataset (filtered_data_unique2)
missing_data_country = filtered_data_unique2[
    filtered_data_unique2[['solar_electricity', 'solar_elec_per_capita', 
                          'oil_electricity', 'oil_elec_per_capita']].isnull().any(axis=1)
]

# Display the country and these features
print("Country missing these features in 2002:")
print(missing_data_country[['country', 'solar_electricity', 'solar_elec_per_capita', 
                           'oil_electricity', 'oil_elec_per_capita']])


## These features were giving some problems to us later in the clustering analysis. So we are trying to visualize the missing data for each country.

Country missing these features in 2002:
Empty DataFrame
Columns: [country, solar_electricity, solar_elec_per_capita, oil_electricity, oil_elec_per_capita]
Index: []


In [851]:
# After the data_2002_2021 filtering and before clustering_features definition

# Extract the last row for each country
latest_records = (
    data_2002_2021.sort_values("year", ascending=False)
    .drop_duplicates(subset=["country"], keep="first")
)

# Analyze missingness in all features
all_features_missingness = latest_records.isnull().sum().reset_index()
all_features_missingness.columns = ["Feature", "Missing Count"]

# Add the percentage of missing values
all_features_missingness["Missing Percentage"] = (
    all_features_missingness["Missing Count"] / len(latest_records) * 100
)

# Sort by missing percentage
all_features_missingness = all_features_missingness.sort_values(
    by="Missing Percentage", ascending=True  # Sort ascending to see complete features first
)

# Display the analysis
print("\nMissingness analysis for all features:")
print(all_features_missingness)

# Show features with zero missing values
complete_features = all_features_missingness[all_features_missingness["Missing Count"] == 0]
print("\nFeatures with complete data:")
print(complete_features)

# Save the analysis
all_features_missingness.to_csv("../datasets/energy/analysis/all_features_missingness_analysis.csv", index=False)


Missingness analysis for all features:
                          Feature  Missing Count  Missing Percentage
0                         country              0            0.000000
26             electricity_demand              0            0.000000
27  electricity_demand_per_capita              0            0.000000
28         electricity_generation              0            0.000000
36         fossil_elec_per_capita              0            0.000000
..                            ...            ...                 ...
23                coal_production            175           84.134615
22           coal_prod_per_capita            175           84.134615
21           coal_prod_change_twh            175           84.134615
20           coal_prod_change_pct            175           84.134615
73        nuclear_cons_change_pct            177           85.096154

[130 rows x 3 columns]

Features with complete data:
                           Feature  Missing Count  Missing Percentage
0       

In [852]:
# Then standardize other numerical features
numerical_features = data_2002_2021.select_dtypes(include=["number"]).columns
numerical_features = numerical_features.difference(["year"])

# Initialize a list to store standardization results for verification
standardization_results = []

# Manually standardize each numerical feature
for feature in numerical_features:
    if data_2002_2021[feature].isnull().all():
        print(f"Feature '{feature}' has only missing values; skipping standardization.")
        continue

    # Calculate mean and standard deviation
    feature_mean = data_2002_2021[feature].mean()
    feature_std = data_2002_2021[feature].std()

    # Avoid division by zero for constant features
    if feature_std == 0:
        print(f"Feature '{feature}' has zero standard deviation; setting standardized values to 0.")
        data_2002_2021[feature] = 0
    else:
        # Standardize the feature
        data_2002_2021[feature] = (data_2002_2021[feature] - feature_mean) / feature_std

    # Verify standardized results
    standardized_mean = data_2002_2021[feature].mean()
    standardized_std = data_2002_2021[feature].std()

    # Append results for verification
    standardization_results.append({
        "Feature": feature,
        "Original Mean": feature_mean,
        "Original StdDev": feature_std,
        "Standardized Mean": standardized_mean,
        "Standardized StdDev": standardized_std,
    })

# Create a DataFrame to display verification results
import pandas as pd
verification_df = pd.DataFrame(standardization_results)
print(verification_df)

# Calculate Non-NA Percentage for Each Country (2002 to 2021)
non_na_by_country_filtered = data_2002_2021.groupby("country").apply(
    lambda group: group.notnull().mean().mean() * 100  # Mean of non-NA values across all columns
).reset_index()

non_na_by_country_filtered.columns = ["Country", "Non-NA Percentage"]

# Sort by Non-NA Percentage in Descending Order
non_na_by_country_filtered_sorted = non_na_by_country_filtered.sort_values(
    by="Non-NA Percentage", ascending=False
)

# Display Results
print("Countries ranked by percentage of available data (2002 to 2021):")
print(non_na_by_country_filtered_sorted)

# Save to CSV for Further Analysis
non_na_by_country_filtered_sorted.to_csv("../datasets/energy/analysis/country_non_na_percentage_2002_to_2021.csv", index=False)

                     Feature  Original Mean  Original StdDev  \
0    biofuel_cons_change_pct      72.892820       372.216566   
1    biofuel_cons_change_twh       0.939974         4.986764   
2    biofuel_cons_per_capita     201.937839       306.135327   
3        biofuel_consumption      12.586782        47.385141   
4    biofuel_elec_per_capita      66.293803       200.907539   
..                       ...            ...              ...   
122     wind_elec_per_capita      85.178035       297.211769   
123         wind_electricity       3.085266        22.493015   
124   wind_energy_per_capita     455.647733       994.304735   
125          wind_share_elec       1.666793         5.234713   
126        wind_share_energy       1.366813         2.733539   

     Standardized Mean  Standardized StdDev  
0         1.005865e-18                  1.0  
1         1.009294e-17                  1.0  
2         9.785719e-17                  1.0  
3         9.951579e-18                  1.0  
4

/var/folders/kb/ldyhzd_s2sg_8j8sr3c0f23h0000gn/T/ipykernel_95786/2722118325.py:45: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



# **Pre-processing of the Dataset**

## Overview
This preprocessing workflow focuses on preparing the **Our World in Data Energy Consumption Dataset** for clustering analysis by filtering relevant records, excluding non-country entities, handling missing values, and standardizing numerical features. These steps ensure the dataset is clean, consistent, and appropriately scaled for machine learning tasks.

---

### **Steps Performed**

#### **1. Data Filtering**
- **Purpose**: To focus on relevant, contemporary, and country-specific data for meaningful clustering results.
- **Actions**:
  - Filtered the dataset to include only records from the year **2002 to 2021**.
  - Excluded non-country entities such as:
    - Continents: Africa, Antarctica, Asia, Europe, North America, South America, and Oceania.
    - Regions: Middle East, Middle Africa, Western Africa, South and Central America, Pacific Islands, and Persian Gulf.
    - Groups: OECD, OPEC, ASEAN, and World.
    - Categories containing the word "income" or "territories."
- **Outcome**:
  - Reduced the dataset from **6659 rows** to **4809 rows** and retained **130 columns**, ensuring only valid country-level data remains.

---

#### **2. Handling Missing Values**
- **Purpose**: To identify and account for features with significant missing data.
- **Actions**:
  - Calculated the percentage of missing values for each column and each country.
  - Retained features with sufficient non-missing data for clustering analysis.
  - Identified countries with excessive missing data and excluded them to maintain data quality.
- **Outcome**:
  - Ensured the dataset includes countries and features with adequate data coverage for meaningful clustering.

---

#### **3. Standardization of Numerical Features**
- **Purpose**: To normalize numerical data, ensuring all features contribute equally to the clustering algorithm.
- **Actions**:
  - Selected all numerical features in the dataset for standardization, excluding the `year` column to preserve its interpretability.
  - For each feature:
    - Calculated its **mean** and **standard deviation**.
    - Transformed the feature to have a **mean of 0** and a **standard deviation of 1**.
    - Verified the results to ensure successful standardization.
- **Key Insights**:
  - Features such as `primary_energy_consumption`, `electricity_demand`, and `carbon_intensity_elec` showed significant variability before standardization.
  - Standardization results confirmed:
    - **Standardized Mean**: Approximated to 0 for all features.
    - **Standardized StdDev**: Exactly 1 for all features.

---

## Insights and Justifications

1. **Temporal Focus**:
   - Filtering records from **2002 to 2021** ensures the analysis reflects modern energy trends and policies.

2. **Exclusion of Non-Country Entities**:
   - Removing regions, continents, and ambiguous entities ensures the dataset is focused on valid country-level data, aligning with the clustering objectives.

3. **Data Completeness**:
   - Retaining features with adequate data ensures meaningful clustering without introducing noise from imputed or overly sparse features.

4. **Standardization**:
   - Standardizing numerical features eliminates the impact of differing scales (e.g., population measured in millions vs. carbon intensity in grams per kWh), ensuring each feature contributes equally to the clustering model.

---

## Summary
The preprocessing steps ensure the dataset is clean, scaled, and ready for clustering analysis. These actions align with the goal of creating robust clusters that accurately represent energy consumption, production, and environmental trends across countries.

# **C. Feature Engineering / Learning from Dataset**

In [853]:
# Define Clustering Features
clustering_features = [
    "primary_energy_consumption",
    "electricity_demand",
    "fossil_electricity",
    "fossil_elec_per_capita",
    "low_carbon_electricity",
    "low_carbon_elec_per_capita",
    "renewables_share_elec",
    "carbon_intensity_elec",
    "energy_per_capita",
    "electricity_demand_per_capita",
    "per_capita_electricity",
    "greenhouse_gas_emissions",
    "year", # Exclude from clustering in processing step
] ## WE ACTUALLY END UP CHANGING THESE FEATURES LATER DUE TO A PIVOT IN OUR CLUSTERING GOAL

# Ensure all selected features exist in the dataset
clustering_features = [feature for feature in clustering_features if feature in data_2002_2021.columns]
print(f"Selected features for clustering: {clustering_features}")

# Filter data for clustering features
data_clustering = data_2002_2021[["country"] + clustering_features].copy()
print(f"Dataset shape after selecting clustering features: {data_clustering.shape}")

# Check for Missing Values
missing_values = data_clustering.isnull().sum()
if missing_values.any():
    print("Warning: Missing values found in clustering features!")
    print(missing_values)
else:
    print("No missing values in the selected features.")

# Calculate the percentage of non-NA data for each clustering feature by country
country_feature_non_na = data_clustering.groupby("country")[clustering_features].apply(
    lambda group: group.notnull().mean() * 100
).reset_index()

# Reshape the data for better visualization (optional)
country_feature_non_na_long = country_feature_non_na.melt(
    id_vars="country", var_name="Feature", value_name="Non-NA Percentage"
)

# Display results
print("Percentage of available data for clustering features by country:")
print(country_feature_non_na_long)

# Define the threshold for minimum non-NA percentage
threshold = 80

# Calculate the average percentage of non-NA data for each country across clustering features
country_feature_non_na_mean = data_clustering.groupby("country")[clustering_features].apply(
    lambda group: group.notnull().mean().mean() * 100
).reset_index()

country_feature_non_na_mean.columns = ["country", "Average Non-NA Percentage"]

# Filter countries above the threshold
countries_above_threshold = country_feature_non_na_mean[
    country_feature_non_na_mean["Average Non-NA Percentage"] >= threshold
]

# Display results
print(f"Countries with data availability above {threshold}%:")
print(countries_above_threshold)

# Filter the dataset to include only countries above the threshold
filtered_data = data_clustering[data_clustering["country"].isin(countries_above_threshold["country"])]
print(f"Filtered dataset shape: {filtered_data.shape}")

Selected features for clustering: ['primary_energy_consumption', 'electricity_demand', 'fossil_electricity', 'fossil_elec_per_capita', 'low_carbon_electricity', 'low_carbon_elec_per_capita', 'renewables_share_elec', 'carbon_intensity_elec', 'energy_per_capita', 'electricity_demand_per_capita', 'per_capita_electricity', 'greenhouse_gas_emissions', 'year']
Dataset shape after selecting clustering features: (4160, 14)
country                           0
primary_energy_consumption       30
electricity_demand                0
fossil_electricity                0
fossil_elec_per_capita            0
low_carbon_electricity            0
low_carbon_elec_per_capita        0
renewables_share_elec             0
carbon_intensity_elec             0
energy_per_capita                30
electricity_demand_per_capita     0
per_capita_electricity            0
greenhouse_gas_emissions          0
year                              0
dtype: int64
Percentage of available data for clustering features by country:

#### Right now we are trying to understand the features that are missing values so that we can select the best ones for clustering

In [854]:
# Analyze missingness for ALL features, but only for countries above threshold
all_features_missingness = filtered_data_unique[
    filtered_data_unique["country"].isin(countries_above_threshold["country"])
].isnull().sum().reset_index()

all_features_missingness.columns = ["Feature", "Missing Count"]

# Add the percentage of missing values
all_features_missingness["Missing Percentage"] = (
    all_features_missingness["Missing Count"] / len(countries_above_threshold) * 100
)

# Sort by missing percentage (ascending to see complete features first)
all_features_missingness = all_features_missingness.sort_values(
    by="Missing Percentage", ascending=True
)

# Display the analysis
print("\nMissingness analysis for all features (only for countries above threshold):")
print(all_features_missingness)

# Show features with zero missing values
complete_features = all_features_missingness[all_features_missingness["Missing Count"] == 0]
print("\nFeatures with complete data for selected countries:")
print(complete_features)

# Save the analysis
all_features_missingness.to_csv("../datasets/energy/analysis/all_features_missingness_analysis_filtered_countries.csv", index=False)


Missingness analysis for all features (only for countries above threshold):
                          Feature  Missing Count  Missing Percentage
0                         country              0            0.000000
26             electricity_demand              0            0.000000
27  electricity_demand_per_capita              0            0.000000
28         electricity_generation              0            0.000000
36         fossil_elec_per_capita              0            0.000000
..                            ...            ...                 ...
23                coal_production            175           84.134615
22           coal_prod_per_capita            175           84.134615
21           coal_prod_change_twh            175           84.134615
20           coal_prod_change_pct            175           84.134615
73        nuclear_cons_change_pct            177           85.096154

[130 rows x 3 columns]

Features with complete data for selected countries:
                  

In [855]:
# Count unique countries in filtered_data_unique
num_countries = len(filtered_data_unique['country'].unique())
print(f"Number of countries in filtered_data_unique: {num_countries}")

# Optionally, display the list of countries
print("\nList of countries:")
for country in sorted(filtered_data_unique['country'].unique()):
    print(country)

Number of countries in filtered_data_unique: 208

List of countries:
Afghanistan
Albania
Algeria
American Samoa
Angola
Antigua and Barbuda
Argentina
Armenia
Aruba
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bermuda
Bhutan
Bolivia
Bosnia and Herzegovina
Botswana
Brazil
British Virgin Islands
Brunei
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Cape Verde
Cayman Islands
Chad
Chile
China
Colombia
Comoros
Congo
Cook Islands
Costa Rica
Cote d'Ivoire
Croatia
Cuba
Cyprus
Czechia
Democratic Republic of Congo
Denmark
Djibouti
Dominica
Dominican Republic
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Eswatini
Ethiopia
Falkland Islands
Faroe Islands
Fiji
Finland
France
French Guiana
French Polynesia
Gabon
Gambia
Georgia
Germany
Ghana
Gibraltar
Greece
Greenland
Grenada
Guadeloupe
Guam
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hong Kong
Hungary
Iceland
India
Indonesia
Iran
Iraq
Ireland
Israel
Italy
Jamaica
Japan
Jord

In [856]:
# visualizing where we are at for all of our data from 2002 - 2021 right now:
print(data_2002_2021.head())

#save the data_2002_2021 to a csv file
data_2002_2021.to_csv("../datasets/energy/analysis/data_2002_2021.csv", index=False)


         country  year iso_code  population       gdp  \
126  Afghanistan  2002      AFG   -0.096841 -0.291503   
127  Afghanistan  2003      AFG   -0.084476 -0.290334   
128  Afghanistan  2004      AFG   -0.077647 -0.289686   
129  Afghanistan  2005      AFG   -0.071200 -0.288107   
130  Afghanistan  2006      AFG   -0.063443 -0.286404   

     biofuel_cons_change_pct  biofuel_cons_change_twh  \
126                      NaN                      NaN   
127                      NaN                      NaN   
128                      NaN                      NaN   
129                      NaN                      NaN   
130                      NaN                      NaN   

     biofuel_cons_per_capita  biofuel_consumption  biofuel_elec_per_capita  \
126                      NaN                  NaN                -0.329972   
127                      NaN                  NaN                -0.329972   
128                      NaN                  NaN                -0.329972   
12

In [857]:
# List of features with no missing values found from analyzing all_features_missingness_analysis.csv
complete_features = [
    'country', 'electricity_generation', 'oil_electricity', 'per_capita_electricity',
    'solar_elec_per_capita', 'fossil_elec_per_capita', 'fossil_electricity',
    'oil_elec_per_capita', 'electricity_demand_per_capita', 'fossil_share_elec',
    'renewables_share_elec', 'net_elec_imports_share_demand', 'renewables_electricity',
    'renewables_elec_per_capita', 'net_elec_imports', 'low_carbon_elec_per_capita',
    'low_carbon_electricity', 'greenhouse_gas_emissions', 'electricity_demand',
    'solar_electricity', 'low_carbon_share_elec', 'year', 'population',
    'oil_share_elec', 'solar_share_elec', 'carbon_intensity_elec'
]

def check_complete_features(data, features):
    # Filter for years 2002-2021
    mask = (data['year'] >= 2002) & (data['year'] <= 2021)
    data_2002_2021 = data[mask]
    
    # Check for NA values
    na_counts = data_2002_2021[features].isna().sum()
    features_with_na = na_counts[na_counts > 0]
    
    # Check for zero values (excluding 'year', 'country')
    numeric_features = [f for f in features if f not in ['year', 'country']]
    zero_counts = data_2002_2021[numeric_features].eq(0).sum()
    features_with_zeros = zero_counts[zero_counts > 0]
    
    print("Features with NA values:")
    if len(features_with_na) > 0:
        print(features_with_na)
    else:
        print("None found")
        
    print("\nFeatures with zero values:")
    if len(features_with_zeros) > 0:
        for feature, count in features_with_zeros.items():
            print(f"{feature}: {count} zeros")
    else:
        print("None found")
        
    # If any zeros found, show which countries and years have them
    if len(features_with_zeros) > 0:
        print("\nDetailed zero value analysis:")
        for feature in features_with_zeros.index:
            zero_mask = data_2002_2021[feature] == 0
            if zero_mask.any():
                print(f"\n{feature} zeros found in:")
                zero_records = data_2002_2021[zero_mask][['country', 'year', feature]]
                print(zero_records)

# Run the check
check_complete_features(data_2002_2021, complete_features)

Features with NA values:
None found

Features with zero values:
None found


In [858]:
# Get list of features that have NA values (excluding 'country' and 'year')
features_with_na = [f for f in complete_features if f not in ['country', 'year']]

# Find rows with any NA values in these features
missing_data_countries = data_2002_2021[
    data_2002_2021[features_with_na].isnull().any(axis=1)
]

# Display the countries, years, and their NA values
print("Countries with missing values:")
print(missing_data_countries[['country', 'year'] + features_with_na])

# Get a summary of which countries have missing data
print("\nSummary of countries with missing data:")
missing_countries = missing_data_countries['country'].unique()
for country in missing_countries:
    years = missing_data_countries[missing_data_countries['country'] == country]['year']
    print(f"{country}: Missing data in years {list(years)}")

Countries with missing values:
Empty DataFrame
Columns: [country, year, electricity_generation, oil_electricity, per_capita_electricity, solar_elec_per_capita, fossil_elec_per_capita, fossil_electricity, oil_elec_per_capita, electricity_demand_per_capita, fossil_share_elec, renewables_share_elec, net_elec_imports_share_demand, renewables_electricity, renewables_elec_per_capita, net_elec_imports, low_carbon_elec_per_capita, low_carbon_electricity, greenhouse_gas_emissions, electricity_demand, solar_electricity, low_carbon_share_elec, population, oil_share_elec, solar_share_elec, carbon_intensity_elec]
Index: []

[0 rows x 26 columns]

Summary of countries with missing data:


In [859]:
# create new csv that has all countries from data_2002_2021 with only certain features
# List of features with no missing values
complete_features = [
    'country','year','population', 'electricity_generation', 'oil_electricity', 'per_capita_electricity',
    'solar_elec_per_capita', 'fossil_elec_per_capita', 'fossil_electricity',
    'oil_elec_per_capita', 'electricity_demand_per_capita', 'fossil_share_elec',
    'renewables_share_elec', 'net_elec_imports_share_demand', 'renewables_electricity',
    'renewables_elec_per_capita', 'net_elec_imports', 'low_carbon_elec_per_capita',
    'low_carbon_electricity', 'greenhouse_gas_emissions', 'electricity_demand',
    'solar_electricity', 'low_carbon_share_elec',
    'oil_share_elec', 'solar_share_elec', 'carbon_intensity_elec'
]

# creating new csv
data_2002_2021_complete_features = data_2002_2021[complete_features]
data_2002_2021_complete_features.to_csv("../datasets/energy/analysis/complete_features_data_2002_2021.csv", index=False)


# **Feature Engineering and Data Completeness Analysis**

## Overview
This section focuses on identifying and selecting features with complete data coverage for our clustering analysis. Through iterative analysis of data completeness, we discovered a larger set of features with no missing values, leading to a pivot in our feature selection strategy.

---

### **Steps Performed**

#### **1. Initial Feature Selection Attempt**
- **Purpose**: To identify key features for clustering based on energy profiles.
- **Initial Approach**:
  - Selected 13 features including energy consumption, demand, and environmental metrics
  - Found issues with missing values (NaN) in these features
- **Outcome**: Realized need to pivot strategy towards using completely populated features

---

#### **2. Complete Features Analysis**
- **Purpose**: To identify features with no missing values across all countries.
- **Steps**:
  - Analyzed missingness for all features in the dataset
  - Identified 26 features with complete data coverage:
    - Generation metrics (e.g., `electricity_generation`, `fossil_electricity`)
    - Per capita metrics (e.g., `per_capita_electricity`, `fossil_elec_per_capita`)
    - Share metrics (e.g., `fossil_share_elec`, `renewables_share_elec`)
    - Environmental indicators (e.g., `carbon_intensity_elec`)
- **Validation**:
  - Verified absence of NaN values
  - Checked for zero values and their distribution
  - Documented countries and years with any data anomalies
- **Results**:
  - We have 208 total countries of data to work with
  - We have 26 features to work with that are completely populated
  - We have 20 years of data to work with from 2002 - 2021

---

#### **3. Data Export and Preparation**
- **Purpose**: To create a clean, complete dataset for clustering analysis.
- **Actions**:
  - Created new dataset containing only complete features
  - Included essential identifiers (`country`, `year`)
  - Exported to "complete_features_data_2002_2021.csv"
- **Outcome**: A robust dataset with no missing values, ready for feature selection and clustering

---

## Insights
- Shifting to complete features provided more reliable data for clustering. It was necessary for us to work only with complete features to retain all data integrity.
- The complete feature set offers diverse metrics across generation, consumption, and environmental impact
- Dataset maintains temporal coverage (2002-2021) while ensuring data completeness
- Next steps will involve selecting the most relevant features from this complete set for clustering

---

## Next Steps
- Review complete features for relevance to clustering objectives
- Remove redundant or unnecessary features
- Prepare final feature set for clustering analysis

# **D. Processing of Dataset**

## **Breakdown of our Clustering Analysis Objective Moving Forward**

### 1. All Available Features After Processing
We have 26 high-quality features with complete data (no missing values or zeros) for years 2000-2021:

```python
complete_features = [
    'country', 'electricity_generation', 'oil_electricity', 'per_capita_electricity',
    'solar_elec_per_capita', 'fossil_elec_per_capita', 'fossil_electricity',
    'oil_elec_per_capita', 'electricity_demand_per_capita', 'fossil_share_elec',
    'renewables_share_elec', 'net_elec_imports_share_demand', 'renewables_electricity',
    'renewables_elec_per_capita', 'net_elec_imports', 'low_carbon_elec_per_capita',
    'low_carbon_electricity', 'greenhouse_gas_emissions', 'electricity_demand',
    'solar_electricity', 'low_carbon_share_elec', 'year', 'population',
    'oil_share_elec', 'solar_share_elec', 'carbon_intensity_elec'
]
```

## **Clustering Goal**

### Primary Objective

Identify distinct patterns in countries' energy systems by focusing on:
- Energy mix composition
- System efficiency and environmental impact
- Development status

### Expected Outcomes

#### **1. High Efficiency, Clean Energy Countries**

**Characteristics:**
- Advanced economies with strong environmental policies
- High per-capita electricity consumption with significant renewable integration
- Low carbon intensity and greenhouse gas emissions

**Focus:** Sustainability and efficiency

#### **2. High Volume, Mixed Energy Countries**

**Characteristics:**
- Large economies with diverse energy sources
- High total electricity generation but moderate per-capita metrics
- Balancing development with sustainability

**Focus:** Economic scale and energy diversity

#### **3. Developing Energy Systems**

**Characteristics:**
- Growing economies with evolving energy infrastructure
- Lower per-capita electricity consumption
- Higher dependency on traditional energy sources like oil

**Focus:** Infrastructure development and energy transition

### Selected Features for Clustering

#### A. Energy Mix Ratios (5 features)

```python
mix_features = [
    'fossil_share_elec',      # Total fossil fuel share
    'renewables_share_elec',  # Total renewables share
    'low_carbon_share_elec',  # Total low carbon share
    'oil_share_elec',         # Oil share in electricity
    'solar_share_elec'        # Solar share in electricity
]
```

**Why:** These features show the energy strategy and priorities of a country, regardless of its size.

**Helps identify:** Progress in energy transition and policy choices.

#### B. Per Capita Metrics (5 features)

```python
per_capita_features = [
    'electricity_demand_per_capita',
    'per_capita_electricity',
    'fossil_elec_per_capita',
    'renewables_elec_per_capita',
    'low_carbon_elec_per_capita'
]
```

**Why:** These metrics indicate the development level and efficiency of a country, independent of its size.

**Helps identify:** Living standards and maturity of energy infrastructure.

#### C. Scale Indicators (3 features)

```python
scale_features = [
    'electricity_generation',
    'electricity_demand',
    'net_elec_imports_share_demand'  # Indicates energy independence
]
```

**Why:** These features show the overall size of the energy system and self-sufficiency.

**Helps identify:** Economic scale and energy independence.

#### D. Environmental Impact and Efficiency (2 features)

```python
environmental_features = [
    'carbon_intensity_elec',  # Emissions per unit of electricity
    'greenhouse_gas_emissions'
]
```

**Why:** These are direct measures of environmental impact.

**Helps identify:** Efficiency of energy production and use.

### Implementation Plan

1. **Data Preparation:**
   - Filter the dataset to include only our selected features

2. **Clustering Analysis:**
   - Use clustering algorithms K-means and K-medians to identify patterns
   - Evaluate the clusters for interpretability and distinctiveness

3. **Validation and Interpretation:**
   - Interpret the results to derive actionable insights

### Features Not Used and Reasons

#### Excluded Features

1. **country and year**
   - **Reason:** These are identifiers and do not provide meaningful information for clustering. They are used for labeling and temporal analysis but not for clustering.

2. **population**
   - **Reason:** While population is important, its effects are already captured in per-capita metrics. Including it directly could skew results due to vast differences in country sizes.

3. **Raw Total Values** (oil_electricity, solar_electricity, etc.)
   - **Reason:** These raw total values are less informative than their corresponding share or per-capita metrics. Shares and per-capita values provide a normalized view that is more comparable across countries.

4. **Redundant Per-Capita Metrics**
   - **Reason:** While these per-capita metrics are useful, they are redundant when we already include broader per-capita metrics like per_capita_electricity and electricity_demand_per_capita. The broader metrics capture overall consumption patterns more effectively.

5. **net_elec_imports**
   - **Reason:** This feature is less informative on its own compared to net_elec_imports_share_demand, which provides a relative measure of energy independence.

6. **electricity_demand**
   - **Reason:** Similar to electricity_generation, but less informative when we already have electricity_demand_per_capita and electricity_generation as scale indicators.

### Summary

The decision to exclude these features is based on the need to:
- Avoid redundancy and focus on the most informative metrics
- Use normalized or relative measures (e.g., shares, per-capita) for better comparability
- Ensure the clustering analysis remains interpretable and focused on key aspects of energy systems
- Avoid the "curse of dimensionality"

**Goal:** Create clusters that are both meaningful and actionable, capturing the essence of each country's energy profile without unnecessary complexity.


In [860]:
# Getting ready to implement clustering tasks
# first we have to get data frame only with features we are using for clustering

# Define the features to use for clustering
selected_features = [
    'fossil_share_elec', 'renewables_share_elec', 'low_carbon_share_elec',
    'oil_share_elec', 'solar_share_elec', 'electricity_demand_per_capita',
    'per_capita_electricity', 'fossil_elec_per_capita', 'renewables_elec_per_capita',
    'low_carbon_elec_per_capita', 'electricity_generation', 'net_elec_imports_share_demand',
    'carbon_intensity_elec', 'greenhouse_gas_emissions'
]

# Retain the country label for interpretation
country_labels = data_2002_2021_complete_features['country']

# Filter the data to include only the selected features
clustering_data = data_2002_2021_complete_features[selected_features]



In [861]:
# Kmeans and PCA from sklearn
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Define the 5-year periods
periods = [(2002, 2006), (2007, 2011), (2012, 2016), (2017, 2021)]

# Initialize a list to store results for each period
all_period_results = []

# Initialize a list to store results for each period
cluster_counts = []

# Initialize a DataFrame to collect all clustering results across all periods
final_cluster_results = pd.DataFrame()

# Initialize a DataFrame to store only country-cluster mappings
country_cluster_results = pd.DataFrame()

# Process each 5-year period
for start_year, end_year in periods:
    # Filter data for the current period
    period_data = data_2002_2021_complete_features[
        (data_2002_2021_complete_features['year'] >= start_year) &
        (data_2002_2021_complete_features['year'] <= end_year)
    ]
    
    # Average the features for each country
    averaged_data = period_data.groupby('country')[selected_features].mean().reset_index()
    
    # Perform K-Means clustering
    kmeans = KMeans(n_clusters=3, random_state=42)
    averaged_data['Cluster'] = kmeans.fit_predict(averaged_data[selected_features])

    # Analyze cluster centroids
    centroids = pd.DataFrame(kmeans.cluster_centers_, columns=selected_features)
    print(f"Cluster centroids for period {start_year}-{end_year}:")
    print(centroids)
    print()
    
    # Analyze feature distribution within each cluster
    for cluster in range(3):
        cluster_data = averaged_data[averaged_data['Cluster'] == cluster]
        print(f"Feature distribution for Cluster {cluster} ({start_year}-{end_year}):")
        print(cluster_data.describe())
        print()
    
    # Perform PCA with 3 components
    pca = PCA(n_components=3)
    pca_result = pca.fit_transform(averaged_data[selected_features])
    averaged_data['PCA1'] = pca_result[:, 0]
    averaged_data['PCA2'] = pca_result[:, 1]
    averaged_data['PCA3'] = pca_result[:, 2]

    # Add a column for the period
    averaged_data['Period'] = f"{start_year}-{end_year}"
    
    # Append the results to the final DataFrame
    final_cluster_results = pd.concat([final_cluster_results, averaged_data], ignore_index=True)

    # Extract country and cluster mapping for a separate CSV
    country_cluster_mapping = averaged_data[['country', 'Cluster', 'Period']]
    country_cluster_results = pd.concat([country_cluster_results, country_cluster_mapping], ignore_index=True)

    # Create a 3D scatter plot with Plotly
    fig = px.scatter_3d(
        averaged_data,
        x='PCA1',
        y='PCA2',
        z='PCA3',
        color='Cluster',
        hover_data=['country'],
        title=f'3D Clusters of Countries Based on Energy Features ({start_year}-{end_year})',
        labels={'PCA1': 'PCA1', 'PCA2': 'PCA2', 'PCA3': 'PCA3'} # Principal Component 1, 2, 3 labels
    )

    # Show the plot
    fig.show()

    # Check the explained variance ratio
    explained_variance = pca.explained_variance_ratio_
    print(f"Explained variance by the first three components: {explained_variance.sum() * 100:.2f}%")
    
    # Store the results
    all_period_results.append((start_year, end_year, averaged_data))

    # Count the number of countries in each cluster
    cluster_count = averaged_data['Cluster'].value_counts().sort_index()
    cluster_counts.append((start_year, end_year, cluster_count))

    # # Visualize the clusters for the current period
    # fig = px.scatter(
    #     averaged_data,
    #     x='PCA1',
    #     y='PCA2',
    #     color='Cluster',
    #     hover_data=['country'],
    #     title=f'Clusters of Countries Based on Energy Features ({start_year}-{end_year})',
    #     labels={'PCA1': 'Principal Component 1', 'PCA2': 'Principal Component 2'}
    # )
    
    # # Show the plot
    # fig.show()

# Print the number of countries in each cluster for each period
for start_year, end_year, counts in cluster_counts:
    print(f"Period {start_year}-{end_year}:")
    for cluster, count in counts.items():
        print(f"  Cluster {cluster}: {count} countries")
    print()
    
# Save the combined results to a CSV file
final_cluster_results.to_csv("../datasets/energy/results/kmeans_clustering_results_by_period.csv", index=False)

# Save the country-cluster mapping to a separate CSV file
country_cluster_results.to_csv("../datasets/energy/results/kmeans_country_cluster_mapping.csv", index=False)

Cluster centroids for period 2002-2006:
   fossil_share_elec  renewables_share_elec  low_carbon_share_elec  \
0           0.709168              -0.690945              -0.709168   
1          -0.636532               0.430315               0.636532   
2          -1.084603               1.037993               1.084603   

   oil_share_elec  solar_share_elec  electricity_demand_per_capita  \
0        0.492493         -0.320632                      -0.090885   
1       -0.660903         -0.322060                       2.673137   
2       -0.543422         -0.321333                      -0.383577   

   per_capita_electricity  fossil_elec_per_capita  renewables_elec_per_capita  \
0               -0.100369                0.160097                   -0.228744   
1                2.785328                1.610530                    1.920667   
2               -0.370209               -0.568900                   -0.042729   

   low_carbon_elec_per_capita  electricity_generation  \
0               

Explained variance by the first three components: 77.15%
Cluster centroids for period 2007-2011:
   fossil_share_elec  renewables_share_elec  low_carbon_share_elec  \
0           0.853299              -0.767313              -0.853299   
1          -1.203096               1.161992               1.203096   
2           0.466194              -0.522085              -0.466194   

   oil_share_elec  solar_share_elec  electricity_demand_per_capita  \
0        1.338103         -0.303214                      -0.214571   
1       -0.515929         -0.299830                      -0.043331   
2       -0.657487         -0.270627                       0.237471   

   per_capita_electricity  fossil_elec_per_capita  renewables_elec_per_capita  \
0               -0.217219                0.058955                   -0.260001   
1               -0.004532               -0.545772                    0.377850   
2                0.213102                0.518779                   -0.161929   

   low_carbon_el

Explained variance by the first three components: 77.80%
Cluster centroids for period 2012-2016:
   fossil_share_elec  renewables_share_elec  low_carbon_share_elec  \
0           0.684130              -0.637305              -0.684130   
1          -1.085172               1.034133               1.085172   
2          -0.980947               0.896631               0.980947   

   oil_share_elec  solar_share_elec  electricity_demand_per_capita  \
0        0.351472         -0.042484                       0.021068   
1       -0.566685         -0.037690                      -0.215561   
2       -0.883089         -0.204163                       3.593540   

   per_capita_electricity  fossil_elec_per_capita  renewables_elec_per_capita  \
0                0.001078                0.308313                   -0.208200   
1               -0.195599               -0.516416                    0.123570   
2                3.730055                0.222937                    4.438347   

   low_carbon_el

Explained variance by the first three components: 79.52%
Cluster centroids for period 2017-2021:
   fossil_share_elec  renewables_share_elec  low_carbon_share_elec  \
0           0.568439              -0.506237              -0.568439   
1          -1.240378               1.160038               1.240378   
2           0.082424              -0.106714              -0.082424   

   oil_share_elec  solar_share_elec  electricity_demand_per_capita  \
0        0.197679          0.696064                       0.004074   
1       -0.671172          0.583203                       0.100144   
2       -0.878974          0.823354                       0.259262   

   per_capita_electricity  fossil_elec_per_capita  renewables_elec_per_capita  \
0               -0.007124                0.269028                   -0.182788   
1                0.102967               -0.501549                    0.476603   
2                0.268172                0.352429                    0.069589   

   low_carbon_el

Explained variance by the first three components: 71.18%
Period 2002-2006:
  Cluster 0: 128 countries
  Cluster 1: 10 countries
  Cluster 2: 70 countries

Period 2007-2011:
  Cluster 0: 68 countries
  Cluster 1: 69 countries
  Cluster 2: 71 countries

Period 2012-2016:
  Cluster 0: 127 countries
  Cluster 1: 76 countries
  Cluster 2: 5 countries

Period 2017-2021:
  Cluster 0: 133 countries
  Cluster 1: 74 countries
  Cluster 2: 1 countries



# **K-means Clustering Implementation Breakdown**

## Overview
This section implements a **K-means** clustering approach on averaged energy-related features across countries and periods. Additionally, **Principal Component Analysis (PCA)** is utilized to reduce dimensionality and enable 3D visualization. Both of these are standard tools from the `sklearn` library, ensuring reliable and well-documented functionality.

---

### **Modules and Functions Explained**

#### **`PCA` Class (from `sklearn.decomposition`)** [2]
- **Purpose**: Principal Component Analysis (PCA) performs dimensionality reduction [3].
- **Key Parameters**:
  - `n_components`: The number of principal components to retain. In this use case, `n_components=3` for effective 3D visualization.
- **Key Attributes**:
  - `explained_variance_ratio_`: Proportion of variance explained by each principal component.
  - `components_`: Principal axes of the original data space (eigenvectors).
- **Key Methods**:
  - `fit_transform(X)`: Fits the PCA model to the dataset `X` and applies the dimensionality reduction.
  - `transform(X)`: Projects new data onto the principal components determined during `fit`.

#### **`KMeans` Class (from `sklearn.cluster`)** [2]
- **Purpose**: The `KMeans` class implements the K-means clustering algorithm, which partitions `n` observations into `k` clusters, minimizing the within-cluster sum of squares [2], [4].
- **Key Parameters**:
  - `n_clusters`: Specifies the number of clusters to form. Here, `n_clusters=3` aligns with the domain-based expectation of three broad country groupings.
  - `random_state`: Controls the randomness for reproducibility. `random_state=42` ensures consistent results across runs.
- **Key Methods**:
  - `fit_predict(X)`: Fits the model to data `X` and returns the cluster labels for each observation.
  - `fit(X)`: Computes cluster centers (centroids).
  - `predict(X)`: Assigns new samples to the previously formed clusters.

---

### **1. Temporal Segmentation of Data**
- **Approach**:
  - The dataset is divided into four 5-year periods: 
    - 2002–2006
    - 2007–2011
    - 2012–2016
    - 2017–2021
  - For each period, the features are averaged by country.
- **Rationale**:
  - Reduces noise and inter-year fluctuations.
  - Provides meaningful temporal snapshots to track how cluster memberships evolve.

---

### **2. K-means Clustering Configuration** [4]

#### **Number of Clusters (n=3)**
- **Rationale**:
  - From domain knowledge, expecting three broad categories of countries based on energy development and policy maturity.
  - Balances granularity and interpretability.

#### **Random State (42)**
- **Purpose**:
  - Ensures reproducibility of clustering results.
  - Allows consistent initialization of cluster centers.
  
---

### **3. Centroid Analysis**
- **What are Centroids?**
  - The cluster centroid is the mean position of all points within a cluster in the feature space.
- **Importance**:
  - Provides insight into the "average" or "typical" profile of countries in each cluster.
  - Allows comparison of cluster characteristics across different periods.

---

### **4. Principal Component Analysis (PCA)** 

#### **Purpose and Benefits** [3]
- **Dimensionality Reduction**:
  - Transforms the original feature set into a smaller set of uncorrelated components.
  - Simplifies visualization by reducing to 3 principal components.
  
- **Explained Variance**:
  - PCA’s `explained_variance_ratio_` quantifies how much information is retained.
  - Typically, retaining around 70–80% of variance in 3 components is sufficient for visual exploration.

#### **Visualization**
- **3D Scatter Plots**:
  - Each point represents a country’s energy profile projected into a 3D PCA space.
  - Points are colored by their cluster assignment, enabling visual validation of clustering performance.

---

### **5. Data Storage and Results**
- **Stored Outputs**:
  - **Clustering Results by Period**: Includes PCA-transformed coordinates, assigned clusters, and country labels.
  - **Country-Cluster Mapping**: Facilitates temporal analysis of how individual countries shift clusters over time.
  - Results stored in CSV files for reproducibility and further analysis.

---

### **Conclusion**
Using **K-means** along with **PCA**:
- We achieve a meaningful segmentation of countries into three clusters.
- PCA aids in validating these clusters by providing an interpretable 3D representation.
- The entire process is built upon well-established `sklearn` modules, ensuring efficiency, reliability, and reproducibility in the clustering pipeline.

In [862]:
# Custom K-Medians Clustering Implementation
class KMedians:
    def __init__(self, n_clusters=3, max_iter=100, random_state=42):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.random_state = random_state
        self.cluster_centers_ = None
        self.labels_ = None

    def fit(self, X):
        np.random.seed(self.random_state)
        # Randomly initialize cluster centers
        initial_indices = np.random.choice(X.shape[0], self.n_clusters, replace=False)
        self.cluster_centers_ = X[initial_indices]

        for iteration in range(self.max_iter):
            # Assign points to the nearest cluster center (L1 distance)
            distances = np.abs(X[:, np.newaxis] - self.cluster_centers_).sum(axis=2)
            self.labels_ = np.argmin(distances, axis=1)

            # Recompute cluster centers as medians
            new_centers = []
            for k in range(self.n_clusters):
                cluster_points = X[self.labels_ == k]
                if len(cluster_points) > 0:
                    median = np.median(cluster_points, axis=0)
                    new_centers.append(median)
                else:
                    # If a cluster has no points, reinitialize it randomly
                    new_centers.append(X[np.random.choice(X.shape[0])])
            new_centers = np.array(new_centers)

            # Check for convergence
            if np.all(self.cluster_centers_ == new_centers):
                break
            self.cluster_centers_ = new_centers

        return self

    def predict(self, X):
        distances = np.abs(X[:, np.newaxis] - self.cluster_centers_).sum(axis=2)
        return np.argmin(distances, axis=1)

# **Breaking down Custom K-medians Implementation**

## Overview
We implemented a custom K-medians clustering algorithm to provide a more robust alternative to K-means. K-medians uses median centers and Manhattan distance, making it less sensitive to outliers in our energy data. [1]

---

### **Class Structure**

#### **Initialization Parameters**
```python
def __init__(self, n_clusters=3, max_iter=100, random_state=42):
```
- `n_clusters`: Number of clusters (default=3)
- `max_iter`: Maximum iterations for convergence (default=100)
- `random_state`: Seed for reproducibility (default=42)
- `cluster_centers_`: Stores median centers
- `labels_`: Stores cluster assignments

---

### **Key Methods**

#### **1. Fit Method**
```python
def fit(self, X):
```
- **Initialization**:
  - Seeds random number generator
  - Randomly selects initial cluster centers from data points
- **Iterative Process**:
  - Computes L1 (Manhattan) distances to cluster centers
  - Assigns points to nearest center
  - Updates centers using median of assigned points
  - Checks for convergence
- **Empty Cluster Handling**:
  - Reinitializes empty clusters with random points
  - Prevents algorithm failure

#### **2. Predict Method**
```python
def predict(self, X):
```
- Assigns new data points to nearest cluster using L1 distance
- Returns cluster labels for input data

---

### **Technical Details**

#### **Distance Calculation**
```python
distances = np.abs(X[:, np.newaxis] - self.cluster_centers_).sum(axis=2)
```
- Uses Manhattan distance (L1 norm)
- More robust to outliers than Euclidean distance
- Computationally efficient implementation using NumPy

#### **Median Center Updates**
```python
median = np.median(cluster_points, axis=0)
```
- Computes element-wise median for each feature
- More robust to extreme values than mean
- Maintains interpretability of cluster centers


In [863]:
# Applying Custom K-Medians Implementation

# Define the 5-year periods
periods = [(2002, 2006), (2007, 2011), (2012, 2016), (2017, 2021)]

# Initialize a list to store results for each period
all_period_results = []

# Initialize a list to store results for each period
cluster_counts = []

# Initialize a DataFrame to store all clustering results across all periods
final_cluster_results = pd.DataFrame()

# Initialize a DataFrame to store only country-cluster mappings
country_cluster_results = pd.DataFrame()

# Process each 5-year period
for start_year, end_year in periods:
    # Filter data for the current period
    period_data = data_2002_2021_complete_features[
        (data_2002_2021_complete_features['year'] >= start_year) &
        (data_2002_2021_complete_features['year'] <= end_year)
    ]
    
    # Average the features for each country
    averaged_data = period_data.groupby('country')[selected_features].mean().reset_index()
    
    # Extract feature values for clustering
    X = averaged_data[selected_features].values

    # Perform K-Medians clustering
    kmedians = KMedians(n_clusters=3, random_state=42)
    kmedians.fit(X)
    averaged_data['Cluster'] = kmedians.labels_

    # Analyze cluster centroids
    centroids = pd.DataFrame(kmedians.cluster_centers_, columns=selected_features)
    print(f"Cluster centroids for period {start_year}-{end_year}:")
    print(centroids)
    print()
    
    # Analyze feature distribution within each cluster
    for cluster in range(3):
        cluster_data = averaged_data[averaged_data['Cluster'] == cluster]
        print(f"Feature distribution for Cluster {cluster} ({start_year}-{end_year}):")
        print(cluster_data.describe())
        print()
    
    # Perform PCA with 3 components for visualization
    pca = PCA(n_components=3)
    pca_result = pca.fit_transform(X)
    averaged_data['PCA1'] = pca_result[:, 0]
    averaged_data['PCA2'] = pca_result[:, 1]
    averaged_data['PCA3'] = pca_result[:, 2]

    # Add a column for the period
    averaged_data['Period'] = f"{start_year}-{end_year}"
    
    # Append the results to the final DataFrame
    final_cluster_results = pd.concat([final_cluster_results, averaged_data], ignore_index=True)

    # Extract country and cluster mapping for a separate CSV
    country_cluster_mapping = averaged_data[['country', 'Cluster', 'Period']]
    country_cluster_results = pd.concat([country_cluster_results, country_cluster_mapping], ignore_index=True)

    # Create a 3D scatter plot with Plotly
    fig = px.scatter_3d(
        averaged_data,
        x='PCA1',
        y='PCA2',
        z='PCA3',
        color='Cluster',
        hover_data=['country'],
        title=f'3D Clusters of Countries Based on Energy Features ({start_year}-{end_year})',
        labels={'PCA1': 'PCA1', 'PCA2': 'PCA2', 'PCA3': 'PCA3'} # Principal Component 1, 2, 3 labels
    )

    # Show the plot
    fig.show()

    # Check the explained variance ratio
    explained_variance = pca.explained_variance_ratio_
    print(f"Explained variance by the first three components: {explained_variance.sum() * 100:.2f}%")
    
    # Store the results
    all_period_results.append((start_year, end_year, averaged_data))

    # Count the number of countries in each cluster
    cluster_count = averaged_data['Cluster'].value_counts().sort_index()
    cluster_counts.append((start_year, end_year, cluster_count))

# Print the number of countries in each cluster for each period
for start_year, end_year, counts in cluster_counts:
    print(f"Period {start_year}-{end_year}:")
    for cluster, count in counts.items():
        print(f"  Cluster {cluster}: {count} countries")
    print()

# Save the combined clustering results to a CSV file
final_cluster_results.to_csv("../datasets/energy/results/kmedians_clustering_results_by_period.csv", index=False)

# Save the country-cluster mapping to a separate CSV file
country_cluster_results.to_csv("../datasets/energy/results/kmedians_country_cluster_mapping.csv", index=False)

Cluster centroids for period 2002-2006:
   fossil_share_elec  renewables_share_elec  low_carbon_share_elec  \
0           0.891384              -0.865440              -0.891384   
1           0.989617              -0.908117              -0.989617   
2          -0.890625               0.854361               0.890625   

   oil_share_elec  solar_share_elec  electricity_demand_per_capita  \
0       -0.654592         -0.322728                       0.582039   
1        1.442025         -0.322728                      -0.485755   
2       -0.798138         -0.322728                      -0.516618   

   per_capita_electricity  fossil_elec_per_capita  renewables_elec_per_capita  \
0                0.632894                1.083485                   -0.234615   
1               -0.492733               -0.396405                   -0.279290   
2               -0.552260               -0.656068                   -0.141562   

   low_carbon_elec_per_capita  electricity_generation  \
0               

Explained variance by the first three components: 77.15%
Cluster centroids for period 2007-2011:
   fossil_share_elec  renewables_share_elec  low_carbon_share_elec  \
0           0.458557              -0.607175              -0.458557   
1           1.016407              -0.935788              -1.016406   
2          -1.062706               1.070794               1.062706   

   oil_share_elec  solar_share_elec  electricity_demand_per_capita  \
0       -0.804427         -0.322728                       0.224159   
1        1.613483         -0.322728                      -0.427320   
2       -0.767832         -0.322728                      -0.561881   

   per_capita_electricity  fossil_elec_per_capita  renewables_elec_per_capita  \
0                0.198042                0.333497                   -0.184045   
1               -0.435645               -0.283903                   -0.280521   
2               -0.559699               -0.659629                   -0.134904   

   low_carbon_el

Explained variance by the first three components: 77.80%
Cluster centroids for period 2012-2016:
   fossil_share_elec  renewables_share_elec  low_carbon_share_elec  \
0           0.970324              -0.888189              -0.970324   
1           0.883989              -0.799013              -0.883989   
2          -0.784973               0.818515               0.784973   

   oil_share_elec  solar_share_elec  electricity_demand_per_capita  \
0       -0.797632         -0.311526                       0.656337   
1        1.277100         -0.321933                      -0.444886   
2       -0.805253         -0.247651                      -0.336172   

   per_capita_electricity  fossil_elec_per_capita  renewables_elec_per_capita  \
0                0.737759                1.426512                   -0.250086   
1               -0.479404               -0.356000                   -0.272696   
2               -0.340372               -0.590577                   -0.085640   

   low_carbon_el

Explained variance by the first three components: 79.52%
Cluster centroids for period 2017-2021:
   fossil_share_elec  renewables_share_elec  low_carbon_share_elec  \
0           0.846167              -0.792478              -0.846167   
1           0.576629              -0.481536              -0.576629   
2          -1.183923               1.126982               1.183923   

   oil_share_elec  solar_share_elec  electricity_demand_per_capita  \
0       -0.827980         -0.216747                       0.494885   
1        0.708329          0.076645                      -0.544275   
2       -0.824395          0.162605                      -0.243425   

   per_capita_electricity  fossil_elec_per_capita  renewables_elec_per_capita  \
0                0.473823                0.831798                   -0.220933   
1               -0.561235               -0.521481                   -0.257269   
2               -0.250972               -0.605946                    0.037044   

   low_carbon_el

Explained variance by the first three components: 71.18%
Period 2002-2006:
  Cluster 0: 46 countries
  Cluster 1: 78 countries
  Cluster 2: 84 countries

Period 2007-2011:
  Cluster 0: 62 countries
  Cluster 1: 72 countries
  Cluster 2: 74 countries

Period 2012-2016:
  Cluster 0: 43 countries
  Cluster 1: 72 countries
  Cluster 2: 93 countries

Period 2017-2021:
  Cluster 0: 53 countries
  Cluster 1: 77 countries
  Cluster 2: 78 countries



# **K-Medians Clustering Implementation Breakdown**

## Overview
This section details our custom implementation of the K-medians clustering algorithm, which offers a more robust alternative to K-means by using medians instead of means for cluster centers. [1]

---

### **1. Custom K-medians Class Implementation**

#### **Class Configuration**
- **Parameters**:
  - `n_clusters=3`: Same clustering structure as K-means
  - `max_iter=100`: Maximum iterations for convergence
  - `random_state=42`: Ensures reproducibility
- **Key Methods**:
  - `fit()`: Trains the clustering model
  - `predict()`: Assigns new data points to clusters

#### **Algorithm Details**
- **Distance Metric**: Uses L1 (Manhattan) distance instead of L2 (Euclidean)
- **Centroid Calculation**: Uses median instead of mean
- **Convergence**: Checks for stability in cluster centers
- **Empty Cluster Handling**: Reinitializes empty clusters randomly

---

### **2. Five-Year Period Analysis**
- **Implementation**: 
  - Same periods as K-means (2002-2006, 2007-2011, 2012-2016, 2017-2021)
- **Advantage**:
  - Median-based approach better handles outliers in temporal data
  - More robust to year-to-year variations

---

### **3. Centroid Analysis**
- **What are K-medians Centroids?**
  - Median point of all samples in a cluster
  - More robust to outliers than mean-based centroids
- **Significance**:
  - Better represents "typical" characteristics in skewed distributions
  - Less influenced by extreme values
  - Provides stable cluster centers over time

---

### **4. Principal Component Analysis (PCA)** [3]
- **Implementation**: 
  - Same as K-means (3 components)
  - Enables visualization of median-based clusters
- **Visualization**:
  - 3D scatter plots show cluster separation
  - Potentially different cluster shapes from K-means
  - Same explained variance analysis

---

### **5. Data Storage and Analysis**
- **Stored Results**:
  - Clustering results by period
  - Country-cluster mappings
  - Median-based centroid characteristics
- **File Outputs**:
  - "kmedians_clustering_results_by_period.csv"
  - "kmedians_country_cluster_mapping.csv"

---

### **6. Key Differences from K-means (this is expanded on in depth in part below)**
- **Distance Calculation**:
  - Uses Manhattan distance (L1 norm)
  - Better suited for high-dimensional data
- **Centroid Updates**:
  - Median calculation instead of mean
  - More computationally intensive but more robust
- **Outlier Handling**:
  - Less sensitive to extreme values
  - May produce more stable clusters

---

## Technical Notes
- Custom implementation required due to lack of built-in K-medians in sklearn
- Potentially more stable results with skewed energy data
- Same visualization and analysis framework as K-means for comparison
- Results stored in similar format for comparative analysis

# **E. Comparative Analysis of K-Means and K-Medians**

In [864]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load previously computed results
kmeans_results = pd.read_csv("../datasets/energy/results/kmeans_clustering_results_by_period.csv")
kmedians_results = pd.read_csv("../datasets/energy/results/kmedians_clustering_results_by_period.csv")

# Define the periods
periods = ["2002-2006", "2007-2011", "2012-2016", "2017-2021"]

for period in periods:
    # Filter the data for the current period
    kmeans_period_data = kmeans_results[kmeans_results['Period'] == period]
    kmedians_period_data = kmedians_results[kmedians_results['Period'] == period]

    # Calculate cluster counts for bar charts
    kmeans_cluster_counts = kmeans_period_data['Cluster'].value_counts().sort_index()
    kmedians_cluster_counts = kmedians_period_data['Cluster'].value_counts().sort_index()

    # Create a figure with 4 subplots (2x2 layout)
    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{'type': 'scene'}, {'type': 'scene'}],
               [{'type': 'bar'}, {'type': 'bar'}]],
        subplot_titles=(f"K-Means Clusters {period}", f"K-Medians Clusters {period}",
                        "K-Means Cluster Counts", "K-Medians Cluster Counts")
    )

    # Add the K-Means 3D scatter plot
    fig.add_trace(
        go.Scatter3d(
            x=kmeans_period_data['PCA1'],
            y=kmeans_period_data['PCA2'],
            z=kmeans_period_data['PCA3'],
            mode='markers',
            marker=dict(
                size=5,
                color=kmeans_period_data['Cluster'],
                colorscale='Viridis',
                showscale=False
            ),
            text=kmeans_period_data['country'],
            hovertemplate="Country: %{text}<br>PCA1: %{x}<br>PCA2: %{y}<br>PCA3: %{z}<extra></extra>"
        ), row=1, col=1
    )

    # Add the K-Medians 3D scatter plot
    fig.add_trace(
        go.Scatter3d(
            x=kmedians_period_data['PCA1'],
            y=kmedians_period_data['PCA2'],
            z=kmedians_period_data['PCA3'],
            mode='markers',
            marker=dict(
                size=5,
                color=kmedians_period_data['Cluster'],
                colorscale='Turbo',
                showscale=False
            ),
            text=kmedians_period_data['country'],
            hovertemplate="Country: %{text}<br>PCA1: %{x}<br>PCA2: %{y}<br>PCA3: %{z}<extra></extra>"
        ), row=1, col=2
    )

    # Add the K-Means bar chart
    fig.add_trace(
        go.Bar(
            x=[f"Cluster {i}" for i in kmeans_cluster_counts.index],
            y=kmeans_cluster_counts.values,
            marker_color='teal'
        ), row=2, col=1
    )

    # Add the K-Medians bar chart
    fig.add_trace(
        go.Bar(
            x=[f"Cluster {i}" for i in kmedians_cluster_counts.index],
            y=kmedians_cluster_counts.values,
            marker_color='orange'
        ), row=2, col=2
    )

    # Define a consistent y-axis range for all subplots
    max_y_value = max(kmeans_cluster_counts.max(), kmedians_cluster_counts.max()) + 5  # Add buffer for aesthetics

    # Update y-axis title and range for both subplots
    fig.update_yaxes(title_text="Number of Countries", range=[0, max_y_value], row=2, col=1)
    fig.update_yaxes(title_text="Number of Countries", range=[0, max_y_value], row=2, col=2)

    # Update layout
    fig.update_layout(
        height=900,
        width=1200,
        title_text=f"Comparison of K-Means and K-Medians Clusters: {period}",
        scene=dict(xaxis_title='PCA1', yaxis_title='PCA2', zaxis_title='PCA3'),
        scene2=dict(xaxis_title='PCA1', yaxis_title='PCA2', zaxis_title='PCA3'),
        showlegend=False
    )

    # Show the figure
    fig.show()


# **Comparative Analysis of K-Means and K-Medians**

## Overview
This section compares the **K-Means** and **K-Medians** clustering methods across several dimensions based on previously computed results. Specifically, we look at:
1. Computational complexity.
2. Sensitivity to outliers.
3. Cluster distributions over time (as represented by the 5-year periods).
4. Explained variance and performance insights.
5. Final recommendation on which method to deploy in production.

[1], [4]

---

### **1. Computational Complexity** [1], [4]

#### **K-Means**
- **Efficiency**: K-Means is computationally efficient as it uses the mean as the centroid, with a mean calculation complexity of 𝑂(𝑛).
- **Description**: K-Means relies on computing means for cluster centers, making it faster and suitable for large datasets. However, it is sensitive to outliers due to the mean calculation.

#### **K-Medians**
- **Efficiency**: K-Medians is computationally more expensive because it uses the median as the centroid, with a median calculation complexity of 𝑂(𝑛log𝑛).
- **Description**: K-Medians is robust to outliers due to its median-based approach but comes at the cost of higher computational complexity, especially for large datasets.

---

### **2. Sensitivity to Outliers**
- **K-Means**: Highly sensitive to outliers, as the mean is affected significantly by extreme values.
- **K-Medians**: Less sensitive to outliers because the median is more robust and does not shift dramatically due to extreme values.

---

### **3. Cluster Distributions Over Time**

The cluster memberships were evaluated for each 5-year period (2002–2006, 2007–2011, 2012–2016, and 2017–2021). The bar charts and 3D PCA plots (side-by-side) provided a visual comparison:

**K-Means Results:**
- **Period 2002–2006**: 
  - Cluster 0: 128 countries 
  - Cluster 1: 10 countries 
  - Cluster 2: 70 countries
- **Period 2007–2011**:
  - Cluster 0: 68 countries 
  - Cluster 1: 69 countries 
  - Cluster 2: 71 countries
- **Period 2012–2016**:
  - Cluster 0: 127 countries 
  - Cluster 1: 76 countries 
  - Cluster 2: 5 countries
- **Period 2017–2021**:
  - Cluster 0: 133 countries 
  - Cluster 1: 74 countries 
  - Cluster 2: 1 country

We observe that K-Means sometimes produces very unbalanced clusters (e.g., one cluster dominating with over 100 countries, another with very few).

**K-Medians Results:**
- **Period 2002–2006**:
  - Cluster 0: 46 countries 
  - Cluster 1: 78 countries 
  - Cluster 2: 84 countries
- **Period 2007–2011**:
  - Cluster 0: 62 countries 
  - Cluster 1: 72 countries 
  - Cluster 2: 74 countries
- **Period 2012–2016**:
  - Cluster 0: 43 countries 
  - Cluster 1: 72 countries 
  - Cluster 2: 93 countries
- **Period 2017–2021**:
  - Cluster 0: 53 countries 
  - Cluster 1: 77 countries 
  - Cluster 2: 78 countries

K-Medians tends to yield a more balanced distribution of countries across clusters for most periods, which can lead to more interpretable and stable cluster groupings.

---

### **4. Explained Variance and Performance**

For both K-Means and K-Medians, the PCA with three components explained about **71.18%** of the variance. This similarity in explained variance means that, from a dimensionality reduction standpoint, both methods captured a similar amount of the data’s structure.

However, when looking at the resultant cluster structures:
- **K-Means** often produced skewed cluster sizes over certain periods, suggesting it might be more influenced by outliers or certain data distributions.
- **K-Medians**, being more robust to outliers, produced more evenly distributed clusters, potentially making the results more stable and meaningful for subsequent analysis or policy-making decisions.

---

### **5. Recommendation for Production Use**

- **When to Choose K-Means**: 
  - If computational speed is crucial and you are confident that your dataset is relatively free of outliers or extreme values.
  - If you prefer the simplicity of interpreting means for cluster centers.

- **When to Choose K-Medians**:
  - If your dataset is likely to contain outliers or skewed distributions.
  - If you value more balanced cluster distributions and robustness in the presence of noisy data.

Given the results:
- **K-Means** struggled in certain periods, producing extremely unbalanced clusters (e.g., only 1 country in a cluster during 2017–2021).
- **K-Medians** yielded more balanced clusters across all periods, and is less sensitive to outliers, potentially offering more stable groupings.

**Final Recommendation**:  
For this particular dataset, **K-Medians** appears to be the more stable and interpretable choice for production due to its resilience to outliers and more even cluster distribution. This can facilitate better downstream analysis and policy implications derived from the clusters.

# **F. Discussion on Ethical Issues**

## Overview
The machine learning tasks conducted on energy-related datasets, particularly clustering countries based on energy consumption, generation, and environmental impact, have important ethical implications. These tasks influence energy policies, economic decisions, and global climate change initiatives. Below, we discuss the key ethical issues associated with this dataset and task. [6]

---

### **1. Data Representation and Bias**
- **Potential Issues**:
  - The dataset may contain inherent biases, such as underrepresentation of low-income or underdeveloped countries. This imbalance could lead to inaccurate clustering or generalizations about energy trends.
  - Developed countries with more detailed and consistent energy data might dominate the analysis, marginalizing countries with limited reporting.
- **Mitigation Strategies**:
  - Perform exploratory analysis to identify gaps in data coverage and account for missing data appropriately.
  - Apply equitable weighting or normalization techniques to ensure fair representation of all countries in the clustering process.

---

### **2. Misinterpretation of Results**
- **Potential Issues**:
  - Clustering results could be misinterpreted as causal relationships rather than observational patterns, leading to misguided energy or climate policies.
  - Simplistic labeling of countries into clusters might ignore socio-economic, cultural, and geographical complexities.
- **Mitigation Strategies**:
  - Clearly communicate that clustering results are descriptive and exploratory, not predictive or causal.
  - Use additional domain expertise to contextualize the findings and avoid overgeneralization.

---

### **3. Socio-Economic Inequities**
- **Potential Issues**:
  - Insights derived from clustering could unintentionally favor developed nations with advanced energy infrastructure, widening existing disparities.
  - Policymakers or organizations might use the results to impose unequal energy or environmental policies on less-resourced countries.
- **Mitigation Strategies**:
  - Highlight socio-economic contexts and limitations of the dataset when presenting clustering results.
  - Ensure that findings are used to identify areas needing support and development, rather than penalizing underperforming regions.

---

### **4. Environmental Justice**
- **Potential Issues**:
  - Clustering results may reveal disparities in energy consumption and greenhouse gas emissions, but failure to act on these insights could perpetuate environmental injustices.
  - High-emitting countries might shift responsibility to others by misusing the analysis.
- **Mitigation Strategies**:
  - Promote transparency in reporting and emphasize accountability for environmental impacts across all countries.
  - Advocate for using these insights to develop fair, sustainable energy policies that address global climate justice.

---

### **5. Data Privacy and Sovereignty**
- **Potential Issues**:
  - While the dataset is aggregated and anonymized, individual country-level data might raise concerns about sovereignty and how the data is used.
  - Countries may object to external entities analyzing and interpreting their energy-related data.
- **Mitigation Strategies**:
  - Ensure that the analysis complies with data-sharing agreements and respects country sovereignty.
  - Engage with stakeholders, including governments and organizations, to encourage collaborative and ethical use of the findings.

---

## Final Thoughts
Analyzing energy systems and clustering countries provides valuable insights into global energy trends, sustainability efforts, and environmental impacts. However, careful attention must be paid to ethical considerations such as data representation, equity, and potential misinterpretation. By addressing these issues proactively, we can ensure that the results are used responsibly to support global energy equity and environmental justice. 

# **G. Bibliography**

[1] S. Atta, "K-Median Clustering Algorithm in Machine Learning and its Python Implementation," Medium, Apr. 10, 2022. [Online]. Available: https://soumenatta.medium.com/k-median-clustering-algorithm-in-machine-learning-and-its-python-implementation-44c9c7a8c413.

[2] Scikit-learn, "Scikit-learn: Machine Learning in Python." Available: https://scikit-learn.org/stable/index.html.

[3] J. Lever, M. Krzywinski, and N. Altman, "Principal component analysis," *Nature Methods*, vol. 14, no. 7, pp. 641–642, Jul. 2017. DOI: https://doi.org/10.1038/nmeth.4346.

[4] W3Schools, "Python Machine Learning - K-Means," W3Schools, 2023. [Online]. Available: https://www.w3schools.com/python/python_ml_k-means.asp.

[5] WhisperingKahuna, “Energy Consumption Dataset by Our World in Data,” Kaggle, [Online]. Available: https://www.kaggle.com/datasets/whisperingkahuna/energy-consumption-dataset-by-our-world-in-data.

[6] B. D. Mittelstadt, P. Allo, M. Taddeo, S. Wachter, and L. Floridi, "The ethics of algorithms: Mapping the debate," *Big Data & Society*, vol. 3, no. 2, Nov. 2016. DOI: https://doi.org/10.1177/2053951716679679.